In [ ]:
#————————————————————

# Name: Azure OpenAI Assistant for Nutritional Values (V1)

# Purpose:


# Company: Allgeier Schweiz AG
# Author: Nicolas Rehder (nrehder@allgeier.ch)
# Create for: SDSC 2024
# Date Created: 22.01.2024
# Last Updated: 22.01.2024
# Python Version: 3.10.4

#General Sources:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant
# https://learn.microsoft.com/en-us/azure/ai-services/openai/assistants-quickstart?tabs=command-line&pivots=programming-language-studio

#Azure Openai Usage:
# https://stackoverflow.com/questions/77986927/in-azure-openai-assistants-when-i-upload-a-file-and-save-it-where-is-that-file-s
# https://techcommunity.microsoft.com/t5/fasttrack-for-azure/strategies-for-optimizing-high-volume-token-usage-with-azure/ba-p/4007751#:~:text=Understanding%20tokens%20and%20limits%20in,generation%2C%20translation%2C%20or%20summarization.

#Additionals:
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models

# Download Python packages (run the below command in terminal if packages have not yet been installed)
# pip install -r C:\Python\sdsc\requirements.txt

#————————————————————

In [12]:
# Import required libraries
import os
import io
import time
from io import StringIO
import json
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
from openai import AzureOpenAI

In [13]:
# Load Azure OpenAI Key and Endpoint. These values can be found within the Azure OpenAI Service resource in portal.azure.com under Keys and Endpoint
load_dotenv(dotenv_path=Path("C:\Python\openai-lab\.venv\.env"))
azure_oai_key = os.environ['AZURE_OPENAI_KEY']
azure_oai_endpoint = os.environ['AZURE_OPENAI_ENDPOINT']

In [14]:
# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key = azure_oai_key,  
    api_version = "2024-02-15-preview",
    azure_endpoint = azure_oai_endpoint
    )

In [15]:
# Load data
path_input = r"C:\Python\openai-lab\data\recipes_short.json" #Change path if required

# send the csv file to the assistant purpose files
response = client.files.create(
  file=open(path_input, "rb"),
  purpose="assistants"
)
print(response)
file__id = response.id

FileObject(id='assistant-YvMMDNtwnUE2laXzfL7D55Oc', bytes=3919070, created_at=1713877817, filename='recipes_short.json', object='file', purpose='assistants', status='processed', status_details=None)


In [16]:
instructions = '''
You are a senior data analyst who will work with data in an csv file.
You have access to a sandboxed environment for writing python code.
The objective is to create a datset for fine-tuning. The dataset must be formatted in the conversational format that is used by the Chat completions API.
An example of the conversational format is available in the EXAMPLES section.
When the user asks you to perform your actions, you will use the provided csv file and examples in the EXAMPLE section.
Execute each of the steps listed below in your ACTIONS section.

EXAMPLES:

{"messages": [{"role": "system", "content": "Act as a head chef and create a flavourful recipe from a list of ingredients"}, {"role": "user", "content": "pork spareribs, soy sauce, fresh garlic, fresh ginger, chili powder, fresh coarse ground black pepper, salt, fresh cilantro leaves, tomato sauce, brown sugar, yellow onion, white vinegar, honey, a.1. original sauce, liquid smoke, cracked black pepper, cumin, dry mustard, cinnamon sticks, orange, juice of, mirin, water"}, {"role": "assistant", "content": "{"name":"backyard style  barbecued ribs","minutes":120,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'south-west-pacific', 'main-dish', 'pork', 'oven', 'holiday-event', 'stove-top', 'hawaiian', 'spicy', 'copycat', 'independence-day', 'meat', 'pork-ribs', 'super-bowl', 'novelty', 'taste-mood', 'savory', 'sweet', 'equipment', '4-hours-or-less']","nutrition":"[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]","n_steps":10,"steps":"['in a medium saucepan combine all the ingredients for sauce#1 , bring to a full rolling boil , reduce heat to medium low and simmer for 1 hour , stirring often', 'rub the ribs with soy sauce , garlic , ginger , chili powder , pepper , salt and chopped cilantro , both sides !', 'wrap ribs in heavy duty foil', 'let stand 1 hour', 'preheat oven to 350 degrees', 'place ribs in oven for 1 hour , turning once after 30 minutes', '3 times during cooking the ribs open foil wrap and drizzle ribs with sauce#1', 'place all the ingredients for sauce#2 in a glass or plastic bowl , whisk well and set aside', 'remove ribs from oven and place on serving platter', 'offer both sauces at table to drizzle over ribs']","description":"this recipe is posted by request and was originaly from chef sam choy's cookbook ","ingredients":"['pork spareribs', 'soy sauce', 'fresh garlic', 'fresh ginger', 'chili powder', 'fresh coarse ground black pepper', 'salt', 'fresh cilantro leaves', 'tomato sauce', 'brown sugar', 'yellow onion', 'white vinegar', 'honey', 'a.1. original sauce', 'liquid smoke', 'cracked black pepper', 'cumin', 'dry mustard', 'cinnamon sticks', 'orange, juice of', 'mirin', 'water']","n_ingredients":22}"}]}
{"messages": [{"role": "system", "content": "Act as a head chef and create a flavourful recipe from a list of ingredients"}, {"role": "user", "content": "lean pork chops, flour, salt, dry mustard, garlic powder, oil, chicken rice soup"}, {"role": "assistant", "content": "{"name":"chicken lickin  good  pork chops","minutes":500,"tags":"['weeknight', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'main-dish', 'pork', 'crock-pot-slow-cooker', 'dietary', 'meat', 'pork-chops', 'equipment']","nutrition":"[105.7, 8.0, 0.0, 26.0, 5.0, 4.0, 3.0]","n_steps":5,"steps":"['dredge pork chops in mixture of flour , salt , dry mustard and garlic powder', 'brown in oil in a large skillet', 'place browned pork chops in a crock pot', 'add the can of soup , undiluted', 'cover and cook on low for 6-8 hours']","description":"here's and old standby i enjoy from time to time. it's from an old newspaper clipping i cut out years ago. very tasty.","ingredients":"['lean pork chops', 'flour', 'salt', 'dry mustard', 'garlic powder', 'oil', 'chicken rice soup']","n_ingredients":7}"}]}


ACTIONS:

1. Read the CSV file.
2. Transform the data and create a jsonl file formatted in the conversational format as shown in the EXAMPLES section.
3. The conversational format has a system, user and assistant text input. 
4. The system text input is always "Act as a head chef and create a flavourful recipe from a list of ingredients".
5. The user text input are the ingredients in the column ingredients of the CSV file.
6. The assistant input are all the columns of the CSV file.
7. Make sure the data transformation has the same format provided by the EXAMPLES section.
3. Prepare the dataset as a jsonl file called table_1 for download by the user.
4. Provide a summary paragraph explaining the preparation of the dataset.

DO NOT:
1. Do not return any images. 
2. Do not return any other file types.
'''

In [17]:
# Create an assistant
assistant = client.beta.assistants.create(
    name = "data analyst assistant",
    instructions = instructions,
    tools = [{"type": "code_interpreter"}],
    model = "gpt-4-1106-preview", #You must replace this value with the deployment name for your model.
    file_ids=[file__id]
)

In [18]:
# Get the file id
fileId = assistant.file_ids[0]
print(assistant)

Assistant(id='asst_71f17K2D4EJmkXs4aWOFERnx', created_at=1713877829, description=None, file_ids=['assistant-YvMMDNtwnUE2laXzfL7D55Oc'], instructions='\nYou are a senior data analyst who will work with data in an json file.\nYou have access to a sandboxed environment for writing python code.\nThe objective is to create a datset for fine-tuning. The dataset must be formatted in the conversational format that is used by the Chat completions API.\nAn example of the conversational format is available in the EXAMPLES section.\nWhen the user asks you to perform your actions, you will use the provided json file and examples in the EXAMPLE section.\nYou will perform the transformation steps to turn the data in the csv file into a dataset compatible for fine-tuning.\nExecute each of the steps listed below in your ACTIONS section.\n\nEXAMPLES:\n\n{"messages": [{"role": "system", "content": "Act as a head chef and create a flavourful recipe from a list of ingredients"}, {"role": "user", "content":

In [19]:
# Create a thread
thread = client.beta.threads.create()

In [20]:
# Add a user prompt to the thread

prompt = "Please execute your ACTIONS on the data stored in the JSON file using the EXAMPLES as reference for the dataset format " + fileId

message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = prompt
)

In [21]:
# Run the Assistant

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [ ]:
while True:
    sec = 30
    # Wait for 30 seconds
    time.sleep(sec)  
    # Retrieve the run status
    run_status = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    # If run is completed, get messages
    if run_status.status == 'completed':
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        # Loop through messages and print content based on role
        for msg in messages.data:
            role = msg.role
            try:
                content = msg.content[0].text.value
                print(f"{role.capitalize()}: {content}")
            except AttributeError:
                # This will execute if .text does not exist
                print(f"{role.capitalize()}: [Non-text content, possibly an image or other file type]")
        break
    elif run.status == "requires_action":
        # handle function calling and continue with the execution
        pass
    elif run.status == "expired" or run.status=="failed" or run.status=="cancelled":
        # run failed, expired, or was cancelled
        break    
    else:
        print("in progress...")

In [19]:
# Functions to read xlsx files from Azure Openai

def read_and_save_file(first_file_id, file_name):    
    # its binary, so read it and then make it a file like object
    file_data = client.files.content(first_file_id)
    file_data_bytes = file_data.read()
    file_like_object = io.BytesIO(file_data_bytes)
    #now read as csv to create df
    returned_data = pd.read_excel(file_like_object)
    returned_data.to_excel(file_name, index=False)
    return returned_data
    # file = read_and_save_file(first_file_id, "analyst_output.csv")
    
def files_from_messages(messages, asst_name):
    first_thread_message = messages.data[0]  # Accessing the first ThreadMessage
    message_ids = first_thread_message.file_ids
    print(message_ids)
    # Loop through each file ID and save the file with a sequential name
    for i, file_id in enumerate(message_ids):
        file_name = f"{asst_name}_output_{i+1}.xlsx"  # Generate a sequential file name
        read_and_save_file(file_id, file_name)
        print(f'saved {file_name}')  

In [ ]:
# extract the file names from the response and retrieve the content
asst_name = 'data_analyst_assistant'        
files_from_messages(messages, asst_name)

In [ ]:
#Clean up

client.beta.assistants.delete(assistant.id)
client.beta.threads.delete(thread.id)
for i in client.files.list():
    client.files.delete(i.id)

In [ ]:

for i in client.beta.assistants.list():
    client.beta.assistants.delete(i.id)
for i in client.beta.threads.list():
    client.beta.threads.delete(i.id)
for i in client.files.list():
    client.files.delete(i.id)

In [25]:
#Test run 50 people

for i in range(50):

    # Load data
    path_input = r"C:\Python\data\openfoodfacts.xlsx" #Change path if required

    # send the csv file to the assistant purpose files
    response = client.files.create(
    file=open(path_input, "rb"),
    purpose="assistants"
    )

    file__id = response.id

    # Create an assistant
    assistant = client.beta.assistants.create(
        name = "data analyst assistant " + str(i),
        instructions = instructions,
        tools = [{"type": "code_interpreter"}],
        model = "gpt-4-1106-preview", #You must replace this value with the deployment name for your model.
        file_ids=[file__id]
    )

    fileId = assistant.file_ids[0]

    thread = client.beta.threads.create()

    prompt = "Please execute your ACTIONS on the data stored in the xlsx file " + fileId

    message = client.beta.threads.messages.create(
        thread_id = thread.id,
        role = "user",
        content = prompt
    )

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
    )
    